# 赛题 [填写赛题编号，如: E题]：[填写赛题名称，如: 交通流量管控]

> **团队成员**: 👨‍💻 @俊宇 (建模与编程), 📊 @队友A (数据分析), ✍️ @队友B (论文写作)
> **文件版本**: V0.1 - 初始数据探索
> **最后更新**: [请在此处填写日期和时间]

---
### **核心思路与模型框架**
*在这个区域，用几句话初步描述你们团队讨论出的解题大方向*
1. **问题一**: 通过轨迹匹配推断转向流量，并用K-Means进行时段划分。
2. **问题二**: 建立排队论模型，利用遗传算法优化信号灯配时方案。
3. ...
---

In [ ]:
# ===================================================================
# 模块一：全局初始化 (Global Initialization)
# ===================================================================

# 导入所有可能用到的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import warnings
import pulp
# ... import other libraries as needed

# 全局美化设置
plt.rcParams['font.sans-serif'] = ['SimHei'] # 解决中文显示问题
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
sns.set_style('whitegrid') # 设置图表风格
warnings.filterwarnings("ignore", category=UserWarning)
print("✅ 指挥中心已启动，所有库准备就绪！(警告过滤器已开启)")

# ===================================================================
# 模块二：数据加载与预处理 (Data Loading & Preprocessing)
# 负责人: @队友A
# ===================================================================

### 2.1 数据加载
* **任务**: 加载题目附件中的所有数据文件。

# 🗺️ Step 1: 数据收集 (Data Collection)
> **🎯 目标**: 加载题目附件中的所有数据文件，**确认数据已成功读入**。
> **🕵️‍♂️ 你的任务**: 
> 1. 将所有附件文件和本Notebook放在**同一个文件夹**下。
> 2. 在下方代码单元格中，**修改`file_path_x`变量为正确的文件名**。
> 3. 按 **Shift+Enter** 执行代码。

In [ ]:
# --- 🔴 请在这里修改文件名 ---
file_path_1 = "附件1.xlsx"
file_path_2 = "附件2.csv"
# file_path_3 = "..." # 如果有更多文件，在此处添加

try:
    # 尝试用Pandas读取数据，你的主要武器是pd.read_excel()和pd.read_csv()
    df1 = pd.read_excel(file_path_1)
    df2 = pd.read_csv(file_path_2, encoding='gbk') # CSV文件有时需要指定编码，如gbk
    # df3 = ...
    print("✅ 所有附件数据加载成功！")
    
    print("\n--- 附件1 数据预览 (前5行) ---")
    display(df1.head())
    print("\n--- 附件2 数据预览 (前5行) ---")
    display(df2.head())
    
except FileNotFoundError as e:
    print(f"❌ 文件未找到，请仔细检查文件名和路径是否完全正确: {e}")
except Exception as e:
    print(f"❌ 数据加载失败，可能是编码问题或其他错误: {e}")

# 🧭 Step 2: 数据探索 (Exploratory Data Analysis - EDA)
> **🎯 目标**: 快速摸清每个数据集的“家底”（**结构、分布、缺失情况**），对数据建立宏观认知。
> **🕵️‍♂️ 你的任务**:
> 1. 逐个执行下方的代码单元格。
> 2. **仔细观察**每个单元格的输出结果。
> 3. 将你的**所有发现和疑惑**，都记录在最后的 **“🕵️‍♂️ 你的【侦察笔记】”** 区域。

In [ ]:
# --- 2.1 检查数据集df1 ---
print("--- 🔬 [df1] 数据结构概览 (info) ---")
df1.info()
print("\n" + "="*50 + "\n")
print("--- 🔬 [df1] 数值型数据统计摘要 (describe) ---")
display(df1.describe())
print("\n" + "="*50 + "\n")
print("--- 🔬 [df1] 类别型数据统计摘要 (describe for objects) ---")
display(df1.describe(include=['object']))

In [ ]:
# --- 2.2 检查数据集df2 ---
print("--- 🔬 [df2] 数据结构概览 (info)---")
df2.info()
print("\n" + "="*50 + "\n")
print("--- 🔬 [df2] 数值型数据统计摘要 (describe)---")
display(df2.describe())
print("\n" + "="*50 + "\n")
print("--- 🔬 [df2] 类别型数据统计摘要 (describe for objects)---")
display(df2.describe(include=['object']))

### 🕵️‍♂️ 你的【侦察笔记】:
> **你的任务**: 这是**最重要**的一步！请像一个真正的侦探一样，在这里写下你从上面所有输出中得到的**初步发现、结论和疑问**。这对后续建模至关重要！
> 
> *   **关于df1**: 
>     *   _例如: “df1共有X行Y列，数据类型均正确，看起来很干净，没有缺失值。”_
> *   **关于df2**:
>     *   _例如: “df2的‘销售额’列存在大量缺失值（缺失XXX个）。"_
>     *   _例如: “df2的‘单价’列，最小值是-99，这明显是异常值，需要处理。”_
>     *   _例如: “df2的‘日期’列是object类型，需要转换成datetime类型才能进行时间序列分析。”_
> *   **初步结论/猜想**:
>     *   _例如: “看起来df1是商品信息表，df2是销售流水表，它们可以通过‘商品ID’关联起来。”_

# 🧹 Step 3: 数据清洗 (Data Cleaning)
> **🎯 目标**: 处理我们在探索阶段发现的**缺失值、异常值、重复值和错误数据类型**。
> **🕵️‍♂️ 你的任务**: 根据你的【侦察笔记】，在下方代码块中，“对症下药”，编写代码清洗数据，并**在【清洗日志】中记录你的每一步操作和理由**。

In [ ]:
# 创建数据集副本进行操作，永远保留原始数据！
df1_cleaned = df1.copy()
df2_cleaned = df2.copy()

print("--- 开始数据清洗流程 ---")

# 示例 1: 处理数据类型错误
# df2_cleaned['日期'] = pd.to_datetime(df2_cleaned['日期'])
# print("✅ ‘日期’列已成功转换为datetime类型。")

# 示例 2: 处理异常值
# 在清洗前，我们先看看有多少异常值
# negative_price_count = (df2_cleaned['单价'] < 0).sum()
# print(f"发现 {negative_price_count} 条单价为负数的异常记录。")
# # 将异常值替换为NaN，稍后与其他缺失值一同处理
# df2_cleaned.loc[df2_cleaned['单价'] < 0, '单价'] = np.nan
# print("✅ 已将负数单价替换为NaN。")

# 示例 3: 处理缺失值
# print("\n--- 清洗前的缺失值统计 ---")
# print(df2_cleaned.isnull().sum())
# # 填充‘销售额’列
# sales_median = df2_cleaned['销售额'].median()
# df2_cleaned['销售额'].fillna(sales_median, inplace=True)
# print("✅ ‘销售额’列缺失值已用中位数填充。")

print("\n--- 🔴 (占位符) 请在此处添加你的数据清洗代码 ---")

In [ ]:
# 创建数据集副本进行操作，永远保留原始数据！
df1_cleaned = df1.copy()
df2_cleaned = df2.copy()

print("--- 开始数据清洗流程 ---")

# 示例 1: 处理数据类型错误
# df2_cleaned['日期'] = pd.to_datetime(df2_cleaned['日期'])
# print("✅ ‘日期’列已成功转换为datetime类型。")

# 示例 2: 处理异常值
# 在清洗前，我们先看看有多少异常值
# negative_price_count = (df2_cleaned['单价'] < 0).sum()
# print(f"发现 {negative_price_count} 条单价为负数的异常记录。")
# # 将异常值替换为NaN，稍后与其他缺失值一同处理
# df2_cleaned.loc[df2_cleaned['单价'] < 0, '单价'] = np.nan
# print("✅ 已将负数单价替换为NaN。")

# 示例 3: 处理缺失值
# print("\n--- 清洗前的缺失值统计 ---")
# print(df2_cleaned.isnull().sum())
# # 填充‘销售额’列
# sales_median = df2_cleaned['销售额'].median()
# df2_cleaned['销售额'].fillna(sales_median, inplace=True)
# print("✅ ‘销售额’列缺失值已用中位数填充。")

print("\n--- 🔴 (占位符) 请在此处添加你的数据清洗代码 ---")

### ✍️ 你的【清洗日志】:
> **你的任务**: 在这里详细记录你的**每一步清洗操作和选择该操作的理由**。这是论文中“数据预处理”章节的核心素材！
> * _**日期格式统一**: 将‘日期’列统一转换为`YYYY-MM-DD`的datetime格式，方便后续按时间进行聚合分析。_
> * _**异常值处理**: 将‘单价’中小于0的值替换为缺失值(NaN)。因为单价不可能是负数，这属于明显的异常数据。后续将与其他缺失值一同处理。_
> * _**缺失值填充**: 对‘销售额’的缺失，我选择了**中位数**填充，因为在EDA阶段发现该数据分布高度右偏，中位数比均值更能抵抗极端值的影响，更能代表一般水平。_

# 🖇️ Step 4: 数据集成 (Data Integration)
> **🎯 目标**: 将多个清洗干净的数据集合并(merge)成一个强大的“**主分析数据集**”。
> **🕵️‍♂️ 你的任务**: 找到合适的关联键(key)，使用`pd.merge()`将df1_cleaned和df2_cleaned合并。

In [ ]:
try:
    # --- 🔴 请在这里修改关联键'key_column_name' ---
    # key_column_name = '商品ID' # 假设这是共同的列名
    # master_df = pd.merge(df2_cleaned, df1_cleaned, on=key_column_name, how='left')
    
    # print("✅ 数据集成功合并！预览合并后的主数据集：")
    # display(master_df.head())
    # print("\n--- 合并后主数据集信息 ---")
    # master_df.info()
    print("🖇️ (占位符) 请在此处编写你的数据集合并代码")
    
except KeyError as e:
    print(f"❌ 合并失败，请检查两个数据表中的关联列名是否完全一致: {e}")

# 🎨 Step 5 (Bonus): 初步可视化洞察
> **🎯 目标**: 对整合后的干净数据，进行初步的可视化，寻找规律。
> **🕵️‍♂️ 你的任务**: 尝试使用`sns.lineplot`, `sns.barplot`等，画1-2张你认为最有价值的图。

In [ ]:
# 示例：分析每日的总销售额趋势
# daily_revenue = master_df.groupby('日期')['销售额'].sum()
# plt.figure(figsize=(15, 6))
# sns.lineplot(data=daily_revenue)
# plt.title("每日总销售额趋势图", fontsize=16)
# plt.show()
print("🎨 (占位符) 请在此处尝试你的第一个可视化分析！")

---
### 🎉 恭喜！你已完成数据侦察与清洗的核心任务！🎉
> **下一步行动**:
> 1. 将这份Notebook文件保存，并命名为`[赛题编号]_Data_Report_V1.ipynb`。
> 2. **在Notion的建模日志中，简要汇报你的核心发现**。
> 3. **通知 @俊宇 (我) 进行Code Review，并讨论后续的建模方向**！
---


# ===================================================================
# --- [阶段性交付点] --- 数据分析师完成工作 ---
# ===================================================================

# ===================================================================
# 模块三：模型构建与求解 (Modeling & Solving)
# 负责人: @俊宇
# ===================================================================

In [ ]:
**(Code) 代码单元格**
```python
# ... 在这里放置你问题一的模型代码 ...
# kmeans = KMeans(n_clusters=3, random_state=42)
# ...```
**(M) Markdown 单元格**
```markdown
### 3.2 [问题二的模型，如：信号灯优化模型]
* **模型思路**: 建立一个以总通行时间最短为目标的线性规划模型...```
**(Code) 代码单元格**
```python
# ... 在这里放置你问题二的模型代码 ...
# prob = pulp.LpProblem(...)
# ...

# ===================================================================
# 模块四：结果分析与可视化 (Result Analysis)
# 负责人: @俊宇 & @队友B
# ===================================================================

### 4.1 核心结果汇总
* **任务**: 将模型求解出的关键结果，以清晰的表格和图表形式展示出来，并存为图片或文件，方便论文引用。
* **(重要: 这里的结果，要和Notion指挥中心的“最终结果汇总区”保持绝对一致！)**

In [ ]:
# ... 在这里放置所有用于生成最终结果表格和图表的代码 ...
# final_result_df.to_excel("问题一结果.xlsx")
# plt.savefig("核心结果图.png", dpi=300)